In [1]:
## Import the libraries

import pandas as pd
import numpy as np
import operator
from collections import OrderedDict
import itertools
import collections
import sklearn
import time

In [2]:
## Load the dataset
df = pd.read_csv('animelists.csv') 

In [3]:
## Overview of the data
df.head()

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:43:35,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-03 10:53:57,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN


#### Creating a smaller dataset for the model and cleaning it

In [ ]:
df = df[~pd.isnull(df['username'])]
good_users = df['username'].value_counts()
good_users = good_users[good_users >20]
good_user_ids = good_users.index.values

import random
user_ids = random.sample(list(good_user_ids), 300)

full_data_filtered = df[df['username'].isin(user_ids)]
full_data_filtered.shape
anime_ids = full_data_filtered['anime_id'].value_counts()
anime_ids = anime_ids[(anime_ids > 20) & (anime_ids <250)]

anime_ids = anime_ids.index.values

full_data_filtered2 = full_data_filtered[full_data_filtered['anime_id'].isin(anime_ids)]
full_data_filtered2['my_score'] = np.where(full_data_filtered2['my_score'] == 0 ,1,full_data_filtered2['my_score'])

In [33]:
## Filtering the columns of interest for the model
df_filtered = full_data_filtered2.loc[:,['username','anime_id','my_score']]

In [34]:
## This is how the final dataset looks like
df_filtered.head()

,username,anime_id,my_score
4566,kioniel,21,9
4567,kioniel,59,9
4568,kioniel,210,9
4569,kioniel,232,2
4570,kioniel,249,7


#### Split the data into train and test

In [36]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_filtered, test_size=0.2, random_state = 42)

#### Eigen Rank Algorithm

In [1]:
def calculate_order_greedy(rating, test, user_id):
    '''Estimate the potential for each anime in the anime list watched by the neighbors
        and rank them using greedy order approach'''
    
    pie = {}
    rank = {}
    rank[9999] = -1
    my_neighbours, Item_list = Neighb(user_id, rating, test)

    for i in Item_list:
 
        second_list = Item_list.copy()
        second_list.remove(i)
        for j in second_list:
 
            pie[i] = preference_func(i, j, user_id, rating,my_neighbours) - preference_func(j, i, user_id, rating,my_neighbours)

    while ((len(Item_list) > 0) and (len(pie) > 0)):
        t = max(pie.items(), key = operator.itemgetter(1))[0]

        rank[t] = len(Item_list)
        Item_list.remove(t)
        pie.pop(t, None)
 
        for k in Item_list:

            pie[k]= pie[k] + preference_func(t,k, user_id, rating,my_neighbours) - preference_func(k,t, user_id, rating,my_neighbours)

    return get_n_longest_values_g(rank)

def get_n_longest_values_g(dictionary, n = 10):
    ''' Outputs the top N anime recommendations for each target user'''
    
    longest_entries = sorted(
        dictionary.items(), key=lambda t: t[1], reverse=True)[:n]
    return [key[0] for key in longest_entries]


def Neighb(user_id, rating, test, cutoff = 10):
    '''Finds the top N neighbors for a target user based on its similarity score with other users. Here N = 10 
        Similarity scores are calculated using kendall tau coefficeint'''
    
    anime_remove = rating[rating['username'] == user_id]['anime_id']
    rating = rating[~rating['anime_id'].isin(anime_remove)]
    all_users = list(rating['username'])

    neighb_score = {}
    Item_list = list(test.loc[test['username'] == user_id, 'anime_id'])
    for u in all_users:

        neighb_score[u] = Kendall_CC(user_id, u, rating)
        
    top_10 = dict(sorted(neighb_score.items(), key=operator.itemgetter(1), reverse=True)[:10])

    return top_10, list(set(Item_list))


def preference_func(A, B, user_id, rating, my_neighbs):
    ''' Calculate the preference function for the neighbors of the target user (user_id) between two animes A and B '''

    pf = 0.00
    num = 0.0
    denom = 0.0
    for key, val in my_neighbs.items():

        if(val!= 0.00):

            mask1 = sum((rating['username'] == key)&(rating['anime_id'] == A))

            mask2 = sum((rating['username'] == key)&(rating['anime_id'] == B))

            if ((mask1 >0 )&( mask2 >0)):
                prod = rating.loc[((rating['username'] == key)&(rating['anime_id'] == A)), 'my_score'].item() - rating.loc[((rating['username'] == key)&(rating['anime_id'] == B)), 'my_score'].item()     

                num = num + prod*val
                denom = denom+val
    if(denom > 0):
        pf = num/denom

    return pf


def indicator_f(x):
    ''' Indicates if the two users have rated anime i more than anime j or vice versa. If yes returns zero else 1'''
    
    if(x <0):
        r = 1
    else:
        r = 0
    return r

def Kendall_CC(u_id, v_id, df):
    '''This function calculates the similarity coefficient between two users using kendall tau'''
    
    S = 0
    
    user_anm_rating = pd.merge(df[df['username'] == u_id], df[df['username'] == v_id], on='anime_id', how='inner', \
                               suffixes={'_U', '_V'})
    
    if(len(user_anm_rating)>5):
        val = 0
        for a in range(len(user_anm_rating)-1):

            e = (user_anm_rating.loc[a,'my_score_U'] - user_anm_rating.loc[a+1,'my_score_U'])*(user_anm_rating.loc[a,'my_score_V'] - user_anm_rating.loc[a+1,'my_score_V'])

            val = val + indicator_f(e)

        S = 1-4*(val/(len(user_anm_rating)*(len(user_anm_rating)-1)))

    return S

In [40]:
## Call the Greedy Order Algorithm for each user in the dataset

df_final = pd.DataFrame()
start_time = time.time()
for i in test['username'].unique():
    my_rank = []
    my_rank = calculate_order_greedy(train, test, i)
    rank_output = pd.DataFrame(v for v in my_rank) 
    rank_output['predict_rank'] = np.arange(1, len(rank_output) + 1)
    rank_output['username'] = i
    df_final = df_final.append(rank_output)
    elapsed_time = time.time() - start_time
    print("time elapsed for one user and all operations", i, elapsed_time, '\n')

In [17]:
## Merge the output of the algorithm with the test data
final_df = test.merge(df_final, how = 'inner', left_on = ['anime_id','username'], right_on = [0,'username'] )
final_df.drop([0],axis=1, inplace = True)

In [21]:
## This is how the final dataframe with original rating and predicted rank looks like
final_df.head()

,username,anime_id,my_score,predict_rank
0,waaaaaaah,21881,1,1
1,negshmeg,8424,1,8
2,Tennou,205,9,7
3,KuroKagami,7311,1,8
4,Boundby,5525,9,9


#### Now we will evaluate our alogirthm using two metrics : Normalized Discounted Cumulative gain (NDCG) and Mean Average Precision(MAP)

#### NDCG Calculation

In [24]:
def ndcg_apm(mydf):
    '''It calculates ndcg for each user and then averages it to get an overall NDCG. This function checks if the
        predcited rank is similar to the actual ranking based on the ratings given by the users'''
    dcg = 0.0
    ndcg = 0.0
    for i in mydf['username'].unique():

        user_df = mydf[mydf['username'] == i]
        ideal_dcg = 0.0
        y_true_sorted = sorted(user_df['my_score'], reverse=True)

        for j in range(user_df.shape[0]):

            ideal_dcg = ideal_dcg + (2 ** y_true_sorted[j] - 1.) / np.log2(j + 2)

        dcg = 0.0
        for kk in range(user_df.shape[0]):

            dcg = dcg + (2**(user_df.iloc[kk, 2]) - 1.0)/ np.log2(1+user_df.iloc[kk, 3])

        ndcg += dcg/ideal_dcg

    res = ndcg/(mydf['username'].nunique())
    return res

In [25]:
## call the NDCG function and print the result
output = ndcg_apm(final_df)
output

#### MAP Calculation

In [27]:
## Mark animes as relevant and recommend based on the threshold 
threshold = 8
final_df['relevant'] = np.where(final_df['my_score'] > threshold, 1,0)
final_df['recommended'] = np.where(final_df['predict_rank'] > threshold, 1,0)

In [28]:
def MAP_k(mydf, k=10):
    ''' This function evaluates that how many of the recommended items to a user are relevant to him
        It then averages MAP value for each user to get an overall measure'''
    
    AP = 0.0
    for i in mydf['username'].unique():

        user_df = mydf[mydf['username'] == i]
        user_df.sort_values('predict_rank', axis=0, inplace=True, ascending=False)
        top_N_items = user_df['recommended'].values[:k+1]

        p_list = np.empty((0,k), int)
        for j in range(len(top_N_items)):
            l = user_df['recommended'].values[:j+1]
            val = np.sum(l)/len(l)
            p_list = np.append(p_list, val)

        sum_val = sum(p_list * top_N_items)
        if(sum(user_df['relevant'] >0)):
            AP = AP + sum_val/sum(user_df['relevant'])
    MAP = AP/mydf['username'].nunique()
    return MAP

In [ ]:
## call the MAP function and print the result
map_output = MAP_k(final_df)
map_output